In [6]:
# импотируем необходимые библиотеки
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
!pip install pytorch-tabnet
from pytorch_tabnet.tab_model import TabNetClassifier
from pytorch_tabnet.callbacks import History
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score


In [7]:
df = pd.read_csv("train.csv")

In [8]:
# выводим общую информацию о датасете
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307000 entries, 0 to 306999
Data columns (total 22 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   request_ts       307000 non-null  int64  
 1   user_id          307000 non-null  object 
 2   referer          307000 non-null  object 
 3   geo_id           306999 non-null  float64
 4   component0       306999 non-null  float64
 5   component1       306999 non-null  float64
 6   component2       306999 non-null  float64
 7   component3       306999 non-null  float64
 8   component4       306999 non-null  float64
 9   component5       306999 non-null  float64
 10  component6       306999 non-null  float64
 11  component7       306999 non-null  float64
 12  component8       306999 non-null  float64
 13  component9       306999 non-null  float64
 14  country_id       306999 non-null  object 
 15  region_id        306999 non-null  object 
 16  timezone_id      306999 non-null  obje

In [9]:
# удаляем пустые строки
df = df.dropna()

In [10]:
df.head()

,request_ts,user_id,referer,geo_id,component0,component1,component2,component3,component4,component5,...,component8,component9,country_id,region_id,timezone_id,browser,browser_version,os,os_version,target
0,1701011363,fb858e8e0a2bec074450eaf94b627fd3,https://9b48ee5/,4799.0,11731.0,4045.0,22213.0,-1184.0,-8992.0,9381.0,...,-899.0,16817.0,c31b4e,470e75,f6155e,Chrome Mobile,119.0.0,Android,10,0.0
1,1700986581,46a5f128fd569c764a92c2eaa788095e,https://9b48ee5/,8257.0,11731.0,4045.0,22213.0,-1184.0,-8992.0,9381.0,...,-899.0,16817.0,c31b4e,44520b,e56e80,Chrome Mobile,111.0.0,Android,10,0.0
2,1701011071,5a74e9ac53ffb21a20cce117c0ad77ba,https://9634fd0/1409e548,3150.0,12498.0,2451.0,10304.0,-6380.0,11608.0,3106.0,...,3347.0,21870.0,c31b4e,616bb9,af47f1,Yandex Browser,20.12.5,Android,11,0.0
3,1700992803,af735816ca19115431ae3d89518c8c91,https://9b48ee5/,2740.0,11731.0,4045.0,22213.0,-1184.0,-8992.0,9381.0,...,-899.0,16817.0,c31b4e,3c9dca,e56e80,Chrome Mobile,119.0.0,Android,10,0.0
4,1701021666,364f0ae0a3f29a685c4fb5bae6033b9a,https://9b48ee5/,4863.0,11731.0,4045.0,22213.0,-1184.0,-8992.0,9381.0,...,-899.0,16817.0,c31b4e,776e76,10b7947,Yandex Browser,18.11.1,Android,4.4.4,0.0


Из данных видим, что есть очевидно лишний столбец user_id, который не поможет при обучении, данные из столбца geo_id требует перевода в тип object, также приведем к целочисленному типу target, приведем к строковому типу данных referer для последующей обработки

In [11]:
df.drop(columns=['user_id'], inplace=True)
df['geo_id'] = df["geo_id"].astype("object")
df['target'] = df["target"].astype("int64")
df = df[df['referer'].apply(lambda x: isinstance(x, str))]

Разделим данные из колонки referer на domain и path - чтобы отдельно обрабатывать информацию об основной странице и отдельно - о вкладках, к которые переходил пользователь. Таким образом, удобнее обрабатывать данные о том, переходил ли пользователь куда-то кроме главной страницы. После со здания двух новых колонок удаляем referer

In [12]:
def split_referer(referer):
    referer = referer[8::]
    parts = referer.split('/', 2)
    domain = parts[0]
    if len(parts) > 1 and parts [1]:
        path = parts[1]
    else:
        path = 'nopath'
    return domain, path

In [13]:
df[['domain', 'path']] = df['referer'].apply(split_referer).tolist()

In [14]:
df.drop(columns=['referer'], inplace=True)

Поскольку категориальные признаки нужно будет переводить в числовые и более правильным видится метод OneHotEncoding (ввиду отсутствия естественного порядка в данных (за исключением разве что browser_version и os_version, но для этого нужна дополнительная предобработка, так как разные версии относятся еще и к разным видам), то необходимо посмотреть на количество возможных новых столбцов. Как видим из статистики, данные нужно сокращать (в противном случае размер датасета становится около 9 Гб и не позволяет обучить модель на имеющихся ресурсах). Я применил следующее: обрезал номера версий ОС и браузеров до основного номера, по признакам 'domain', 'path', 'region_id', 'geo_id', 'browser_version', 'timezone_id' - редкие категории (количество которых менее 100) и по признаку 'os_version' (если количество менее 30) заменил на искуствнное значение 0

In [15]:
columns_to_check = ['domain', 'path', 'country_id', 'geo_id', 'region_id', 'timezone_id', 'browser_version', 'os', 'os_version']
unique_counts = df[columns_to_check].agg('nunique')
print(unique_counts)

domain              3980
path               76894
country_id            12
geo_id              1824
region_id            250
timezone_id           60
browser_version     1109
os                     7
os_version           201
dtype: int64


In [16]:
col_for_del = ['browser_version', 'os_version']
for column in col_for_del:
    df[column] = df[column].astype(str).str.split('.').str[0]

In [17]:
for column in columns_to_check:
    print(f"Статистика по столбцу '{column}':")
    value_counts = df[column].value_counts()
    print(value_counts)
    print("-" * 30)

Статистика по столбцу 'domain':
domain
72879b4    29235
6a81948    26752
8807153    15669
9b08d64    14254
9f1218f    12131
           ...  
9bb16a6        1
9f4a150        1
a0fdae6        1
9485311        1
77b579d        1
Name: count, Length: 3980, dtype: int64
------------------------------
Статистика по столбцу 'path':
path
nopath      112951
16658dd6      1829
172507b2      1787
175de82a      1278
1458ef49      1050
             ...  
15d7bc32         1
15349efa         1
12ad935b         1
16937fb2         1
16a356f9         1
Name: count, Length: 76894, dtype: int64
------------------------------
Статистика по столбцу 'country_id':
country_id
c31b4e     269884
121db33     15945
af12ca       5698
b98648       4620
1234f1d      4234
ac5671       3629
110628b       905
e37756        826
eba88b        772
103bf7d       286
122be0f       178
ff9306         22
Name: count, dtype: int64
------------------------------
Статистика по столбцу 'geo_id':
geo_id
3663.0    53999
2521.0    23

In [18]:
def rare_100(df, columns):
    for col in columns:
        value_counts = df[col].value_counts()
        rare_values = value_counts[value_counts <= 100].index
        df[col] = df[col].replace(rare_values, 0)
    return df

def rare_os(df, columns):
    for col in columns:
        value_counts = df[col].value_counts()
        rare_values = value_counts[value_counts <= 30].index
        df[col] = df[col].replace(rare_values, 0)
    return df

In [19]:
df = rare_100(df, ['domain', 'path', 'region_id', 'geo_id', 'browser_version', 'timezone_id'])
df = rare_os(df, ['os_version'])

<ipython-input-18-d5f553256292>:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].replace(rare_values, 0)


In [20]:
for column in columns_to_check:
    print(f"Статистика по столбцу '{column}':")
    value_counts = df[column].value_counts()
    print(value_counts)
    print("-" * 30)

Статистика по столбцу 'domain':
domain
0          30018
72879b4    29235
6a81948    26752
8807153    15669
9b08d64    14254
           ...  
6765856      102
84614fa      102
ad3dab1      102
6606efb      101
862a53d      101
Name: count, Length: 246, dtype: int64
------------------------------
Статистика по столбцу 'path':
path
0           151599
nopath      112951
16658dd6      1829
172507b2      1787
175de82a      1278
             ...  
15436b9b       104
127baebd       103
17d23922       103
142ad557       102
12fd3182       102
Name: count, Length: 177, dtype: int64
------------------------------
Статистика по столбцу 'country_id':
country_id
c31b4e     269884
121db33     15945
af12ca       5698
b98648       4620
1234f1d      4234
ac5671       3629
110628b       905
e37756        826
eba88b        772
103bf7d       286
122be0f       178
ff9306         22
Name: count, dtype: int64
------------------------------
Статистика по столбцу 'geo_id':
geo_id
3663.0    53999
0.0       24487

Для категориальных признаков применяем OneHotEncoding

In [21]:
features_to_encode = ['domain', 'path', 'country_id', 'region_id', 'os', 'os_version', 'browser', 'timezone_id', 'browser_version']

In [22]:
df = pd.get_dummies(df, columns=features_to_encode, prefix=features_to_encode)

In [23]:
def convert_bool_columns_to_int(df):
    for column in df.columns:
        if df[column].dtype == 'bool':
            df[column] = df[column].astype(int)
    return df

df = convert_bool_columns_to_int(df)

Создаем отдельные датафреймы с обучающими данными и с целевым признаком. Разделяем данные на обучающую, валидационную и тестовую выборки, преобразуем их к формату, читаемому моделью TabNet

In [24]:
df.to_csv('/content/dfdot.csv', sep=';', encoding='utf-8')
df.to_csv('/content/dfnedot.csv', sep=',', encoding='utf-8')

In [25]:
X = df.drop('target', axis=1)
y = df['target']

In [26]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.4, random_state=42)

In [27]:
X_train = X_train.values
X_valid = X_valid.values
X_test = X_test.values
y_train = y_train.values
y_valid = y_valid.values
y_test = y_test.values

Обучение проводилось с разными вариациями гиперпараметров (в частности, max_epochs, batch_size, num_workers). Ниже представлена итоговая версия, которая показала лучший результат (все значения оставлены дефолтными кроме количества эпох)

In [ ]:
clf = TabNetClassifier()
clf.fit(
  X_train, y_train,
  eval_set=[(X_valid, y_valid)],
  max_epochs=35,
  callbacks=[]
)

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.6631  | val_0_auc: 0.69574 |  0:02:06s
epoch 1  | loss: 0.58673 | val_0_auc: 0.68392 |  0:04:12s
epoch 2  | loss: 0.52207 | val_0_auc: 0.69173 |  0:06:17s
epoch 3  | loss: 0.49375 | val_0_auc: 0.72337 |  0:08:27s
epoch 4  | loss: 0.47559 | val_0_auc: 0.85238 |  0:10:34s
epoch 5  | loss: 0.46836 | val_0_auc: 0.86363 |  0:12:44s
epoch 6  | loss: 0.45458 | val_0_auc: 0.86973 |  0:14:54s
epoch 7  | loss: 0.44524 | val_0_auc: 0.87244 |  0:17:02s
epoch 8  | loss: 0.44053 | val_0_auc: 0.87513 |  0:19:16s
epoch 9  | loss: 0.4402  | val_0_auc: 0.87602 |  0:21:27s
epoch 10 | loss: 0.43408 | val_0_auc: 0.87766 |  0:23:32s
epoch 11 | loss: 0.43554 | val_0_auc: 0.87798 |  0:25:45s
epoch 12 | loss: 0.43127 | val_0_auc: 0.87845 |  0:27:52s
epoch 13 | loss: 0.42809 | val_0_auc: 0.87962 |  0:29:57s
epoch 14 | loss: 0.4297  | val_0_auc: 0.87655 |  0:32:08s
epoch 15 | loss: 0.42636 | val_0_auc: 0.8799  |  0:34:19s
epoch 16 | loss: 0.42442 | val_0_auc: 0.88102 |  0:36:24s
epoch 17 | los

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Получаем историю обучения и строим график

In [ ]:
history = clf.history
train_loss = history['train_loss']
valid_loss = history['valid_0_loss']
valid_accuracy = history['valid_0_accuracy']

In [ ]:
epochs = range(1, len(train_loss) + 1)

plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.plot(epochs, train_loss, 'b', label='Training loss')
plt.plot(epochs, valid_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Эпохи')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(epochs, valid_accuracy, 'g', label='Validation accuracy')
plt.title('Validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
preds = clf.predict(X_test)
accuracy = accuracy_score(y_test, preds)
print(f"Accuracy на тестовых данных: {accuracy:.4f}")

Accuracy на тестовых данных: 0.7980


In [ ]:
# вычисляем важность признаков, основываясь на масках внимания TabNet
def get_feature_importances(model, x, feature_names):
    try:
        att_masks = model.explain(x)
    except Exception as e:
        print(f"Ошибка при вызове model.explain(): {e}")
        return None

    # Sum the importance of each feature
    feature_importances = np.sum(np.mean(att_masks, axis=0), axis=0)

    # Create a pandas DataFrame for easier handling and visualization
    feature_importance_df = pd.DataFrame({
        'feature': feature_names,
        'importance': feature_importances
    })
    feature_importance_df = feature_importance_df.sort_values(by='importance', ascending=False).reset_index(drop=True)
    return feature_importance_df

In [ ]:
feature_importance_df = get_feature_importances(clf, X_test, ['os', 'os_version'])

In [ ]:
# Визуализация важности признаков (опционально):
def plot_feature_importances(feature_importance_df, top_n=None, title="Feature Importances"):
    """
    Строит график важности признаков.

    Args:
        feature_importance_df (pandas.DataFrame): DataFrame с важностью признаков.
        top_n (int, optional): Количество топ признаков для отображения. Defaults to None.
        title (str, optional): Заголовок графика. Defaults to "Feature Importances".
    """
    if top_n:
        feature_importance_df = feature_importance_df.head(top_n)

    plt.figure(figsize=(10, 6))
    sns.barplot(x="importance", y="feature", data=feature_importance_df, palette="viridis")
    plt.title(title)
    plt.xlabel("Importance")
    plt.ylabel("Feature")
    plt.tight_layout()
    plt.show()

# Визуализируем
if feature_importance_df is not None:
    plot_feature_importances(feature_importance_df, top_n=10, title="Top 10 Feature Importances")
else:
    print("Не удалось получить карту важности признаков")